### Measuring the correlation between categorical variables Provider Tenure and  Medication Given

This notebook analyzes and describes relationship that exists( or not exist) between provider tenure and EMS treatement( Medication Given) to the patient. The script reads (primarily) the intermediate dataset -- 'MedicationsPatients' preapared by our team based on datasets provided by our project partner.

Initially, in this classification situation, catogorical target variable 'Medication_Given' and categorical predicator 'Provider's Tenure'(in months & years), are analyzed and the strength of relationship between them is measured using <B>Chi-square test<B>.
    
Then the more details analysis of the relationship between provider's tenure and Medication Given is done including various graphical/visual representations

<b>Chi-square Test of Independence:<b>
<p>The chi-square test of independence is used to determine whether there is an association between two or more categorical variables. In our case, we would like to test whether the Tenure of the provider has any association with Medication they administered.

##### This note book uses Intermediate Datasets -- MedicationsPatients for analytics

In [1]:
# import the libraries needed
import pandas as pd
import numpy as np
import os, time

# Used for visualizations
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
print(os.getcwd())
readStart=time.time()

## Reading dataset
columnsToUse=['PatientId','PatientGender','PatientGenderCode',
            'FRDPersonnelID','FRDPersonnelGender','ProviderGenderCode',
            'Medication_Given_RXCUI_Code','Medication_Given_Description','TenureMonths','DispatchTime', 'FRDPersonnelStartDate']

dfMedPatient = pd.read_csv ('../data/02_intermediate/MedicationsPatients-20210225-ems-raw-v04.csv',usecols=columnsToUse)[columnsToUse]

# Stop the clock and calculate read time
readStop=time.time()
readTime=readStop-readStart
readMin=np.floor(readTime/60)
readSec=np.floor(readTime-(readMin*60))
print("The file was read in {0:.0f} minutes and {1:.0f} seconds.".format(readMin,readSec))

print(dfMedPatient.shape)

C:\github_anup\ems-analytics\notebooks
The file was read in 0 minutes and 0 seconds.
(62881, 11)


In [3]:
# list the columns in the dataframe
dfMedPatient.columns

Index(['PatientId', 'PatientGender', 'PatientGenderCode', 'FRDPersonnelID',
       'FRDPersonnelGender', 'ProviderGenderCode',
       'Medication_Given_RXCUI_Code', 'Medication_Given_Description',
       'TenureMonths', 'DispatchTime', 'FRDPersonnelStartDate'],
      dtype='object')

##### Remove records that have nulls in the Medication_Given_RXCUI_Code and Medication_Given_Description Column for this question

In [4]:
#Calculate the number of null values per columns
dsNumNull=dfMedPatient.isnull().sum()
dsNumNull.name='Number Rows With Nulls'
print(dsNumNull)

PatientId                        0
PatientGender                   32
PatientGenderCode                0
FRDPersonnelID                   0
FRDPersonnelGender               0
ProviderGenderCode               0
Medication_Given_RXCUI_Code      6
Medication_Given_Description     6
TenureMonths                     0
DispatchTime                     0
FRDPersonnelStartDate            0
Name: Number Rows With Nulls, dtype: int64


In [5]:
## Records with Null values in columns: Medication_Given_RXCUI_Code,Medication_Given_Description 
dfMedPatient[dfMedPatient.Medication_Given_Description.isnull()]

,PatientId,PatientGender,PatientGenderCode,FRDPersonnelID,FRDPersonnelGender,ProviderGenderCode,Medication_Given_RXCUI_Code,Medication_Given_Description,TenureMonths,DispatchTime,FRDPersonnelStartDate
1313,500147,Male,2,80D2C99E-9E01-E211-B5F5-78E7D18CFD3C,Male,2,NaN,NaN,245,2018-01-22 16:03:54,1997-08-18
7638,600712,Female,1,A5D5C99E-9E01-E211-B5F5-78E7D18CFD3C,Female,1,NaN,NaN,82,2018-05-17 20:59:30,2011-07-11
9938,637931,Female,1,4B7A3FE8-2D9E-E311-B3F8-AE437D699481,Male,2,NaN,NaN,53,2018-06-27 18:46:34,2014-01-13
20122,804502,Male,2,FAD5C99E-9E01-E211-B5F5-78E7D18CFD3C,Male,2,NaN,NaN,93,2019-01-02 14:04:02,2011-03-28
29899,969263,Male,2,35CA45A3-9C25-4E4D-957F-2EA9E40B5974,Male,2,NaN,NaN,40,2019-07-02 16:04:48,2016-02-08
40585,1149724,Male,2,4BD3C99E-9E01-E211-B5F5-78E7D18CFD3C,Male,2,NaN,NaN,179,2020-01-13 10:56:09,2005-02-05


In [6]:
## Removing rows with Null values
dfMedPatient2=dfMedPatient.dropna(axis=0,subset=['Medication_Given_Description'])
print("dfMedPatient  -> {}\ndfMedPatient2 -> {}".format(len(dfMedPatient),len(dfMedPatient2)))

dfMedPatient  -> 62881
dfMedPatient2 -> 62875


In [7]:
## Assining cleaned dataframe for further analysis, after dropping 6 records
dfMedPatient = dfMedPatient2

In [8]:
#Confirming Given Medication Names
print('Medication Given( Unique ) =\n '+str(dfMedPatient.Medication_Given_Description.unique()))

Medication Given( Unique ) =
 ['Oxygen' 'Ondansetron (Zofran)' 'Normal saline'
 'Acetylsalicylic Acid (Aspirin, ASA) ' 'Fentanyl Citrate (Sublimaze) '
 'Nitroglycerin' 'Sodium Bicarbonate (8.4%) ' 'Dextrose 50 % (D50)'
 'Ipratropium Bromide(Atrovent)' 'Albuterol (Ventolin) '
 'Dextrose 10 % (D10)' 'Naloxone (Narcan)' 'Midazolam (Versed)'
 'Ketamine (Ketalar)' 'Diphenhydramine (Benadryl)'
 'Methylprednisolone (Solu-Medrol) ' 'Magnesium Sulfate (50%) '
 'Glucose (Oral)' 'Epinephrine 0.1 MG/ML (1:10,000)'
 'Amiodarone (Cordarone)' 'Glucagon (Glucagen)'
 'Morphine (Morphine Sulfate) ' 'Atropine (Atropine Sulfate) '
 'Adenosine (Adenocard)' 'Epinephrine 1 MG/ML (1:1,000)'
 'Lidocaine (2%) (Xylocaine) ' 'Tranexamic Acid (TXA)'
 'Cyanokit (Hydroxocobalamin)' 'Norepinephrine (Levophed)'
 'Calcium Chloride (10%) ' 'Tetracaine'
 'Sodium Bicarbonate (4.2%) (Infant)']


In [9]:
#Checking Medication Given Description count
print('Medication Given Description Count = '+str(dfMedPatient.Medication_Given_Description.nunique()))
##dfMedPatient.Medication_Given_Description.nunique()

Medication Given Description Count = 32


In [10]:
## Counting medications given
dfMedPatient["Medication_Given_Description"].value_counts()

Oxygen                                  10707
Normal saline                            7552
Ondansetron (Zofran)                     6855
Fentanyl Citrate (Sublimaze)             6577
Epinephrine 0.1 MG/ML (1:10,000)         6189
Acetylsalicylic Acid (Aspirin, ASA)      4349
Albuterol (Ventolin)                     3837
Ipratropium Bromide(Atrovent)            2817
Nitroglycerin                            2721
Naloxone (Narcan)                        2056
Midazolam (Versed)                       1395
Dextrose 10 % (D10)                      1187
Glucose (Oral)                            999
Methylprednisolone (Solu-Medrol)          981
Diphenhydramine (Benadryl)                915
Dextrose 50 % (D50)                       866
Ketamine (Ketalar)                        642
Epinephrine 1 MG/ML (1:1,000)             565
Adenosine (Adenocard)                     393
Atropine (Atropine Sulfate)               313
Amiodarone (Cordarone)                    294
Glucagon (Glucagen)               

<b>Add personnel start year and personnel start year_month column to the dataframe</b>

In [11]:
## Adding personnel start year 
dfMedPatient['PersonelStartYear'] = pd.DatetimeIndex(dfMedPatient['FRDPersonnelStartDate']).year

## Adding personnel start year_month
##Note : #if the date is in datetime format, we can also extract the day/month/year using the to_period function  using 'D', 'M', 'Y' as inputs
dfMedPatient['PersonelStartMonthYear'] = pd.to_datetime(dfMedPatient['FRDPersonnelStartDate']).dt.to_period('M')

## Adding MedicationGivenYear 
dfMedPatient['MedicationGivenYear'] = pd.DatetimeIndex(dfMedPatient['DispatchTime']).year

creating output to generate an MedicationDescriptionOrder to be used in plots later.

In [12]:
#Put the value_counts into a dataframe for later use
value_countsMedGiven=dfMedPatient['Medication_Given_Description'].value_counts()
df_MedicationGivenDescription=pd.DataFrame(value_countsMedGiven)
df_MedicationGivenDescription.reset_index(inplace=True)
df_MedicationGivenDescription.columns=["Medication_Given_Description","Count"]
df_MedicationGivenDescription.set_index(keys=["Medication_Given_Description"])
df_MedicationGivenDescription

,Medication_Given_Description,Count
0,Oxygen,10707
1,Normal saline,7552
2,Ondansetron (Zofran),6855
3,Fentanyl Citrate (Sublimaze),6577
4,"Epinephrine 0.1 MG/ML (1:10,000)",6189
5,"Acetylsalicylic Acid (Aspirin, ASA)",4349
6,Albuterol (Ventolin),3837
7,Ipratropium Bromide(Atrovent),2817
8,Nitroglycerin,2721
9,Naloxone (Narcan),2056


In [13]:
#Create the MedicationGivenOrder array from the newly created dataframe
MedicationGivenOrder=np.array(df_MedicationGivenDescription["Medication_Given_Description"])
MedicationGivenOrder

array(['Oxygen', 'Normal saline', 'Ondansetron (Zofran)',
       'Fentanyl Citrate (Sublimaze) ',
       'Epinephrine 0.1 MG/ML (1:10,000)',
       'Acetylsalicylic Acid (Aspirin, ASA) ', 'Albuterol (Ventolin) ',
       'Ipratropium Bromide(Atrovent)', 'Nitroglycerin',
       'Naloxone (Narcan)', 'Midazolam (Versed)', 'Dextrose 10 % (D10)',
       'Glucose (Oral)', 'Methylprednisolone (Solu-Medrol) ',
       'Diphenhydramine (Benadryl)', 'Dextrose 50 % (D50)',
       'Ketamine (Ketalar)', 'Epinephrine 1 MG/ML (1:1,000)',
       'Adenosine (Adenocard)', 'Atropine (Atropine Sulfate) ',
       'Amiodarone (Cordarone)', 'Glucagon (Glucagen)',
       'Magnesium Sulfate (50%) ', 'Sodium Bicarbonate (8.4%) ',
       'Tranexamic Acid (TXA)', 'Lidocaine (2%) (Xylocaine) ',
       'Morphine (Morphine Sulfate) ', 'Norepinephrine (Levophed)',
       'Calcium Chloride (10%) ', 'Tetracaine',
       'Cyanokit (Hydroxocobalamin)',
       'Sodium Bicarbonate (4.2%) (Infant)'], dtype=object)

In [14]:
#Confirming unique Medication_Given_RXCUI_Code count
print('Medication_Given_RXCUI_Code Count = '+str(dfMedPatient.Medication_Given_RXCUI_Code.nunique()))

Medication_Given_RXCUI_Code Count = 32


Removing 'Oxygen' and 'Normal saline' from 'MedicationGivenOrder' array for later sorting /indexing purpose


In [15]:
#Removing MedicationGivenOrder 'Oxygen' and 'Normal saline' from 'MedicationGivenOrder' numpy array using 'numpy.delete(array, index)'

## index : 0='Oxygen' ; 1 ='Normal saline'
index = [0, 1]
MedicationGivenOrderWithoutOxySaline= np.delete(MedicationGivenOrder,index)
MedicationGivenOrderWithoutOxySaline

array(['Ondansetron (Zofran)', 'Fentanyl Citrate (Sublimaze) ',
       'Epinephrine 0.1 MG/ML (1:10,000)',
       'Acetylsalicylic Acid (Aspirin, ASA) ', 'Albuterol (Ventolin) ',
       'Ipratropium Bromide(Atrovent)', 'Nitroglycerin',
       'Naloxone (Narcan)', 'Midazolam (Versed)', 'Dextrose 10 % (D10)',
       'Glucose (Oral)', 'Methylprednisolone (Solu-Medrol) ',
       'Diphenhydramine (Benadryl)', 'Dextrose 50 % (D50)',
       'Ketamine (Ketalar)', 'Epinephrine 1 MG/ML (1:1,000)',
       'Adenosine (Adenocard)', 'Atropine (Atropine Sulfate) ',
       'Amiodarone (Cordarone)', 'Glucagon (Glucagen)',
       'Magnesium Sulfate (50%) ', 'Sodium Bicarbonate (8.4%) ',
       'Tranexamic Acid (TXA)', 'Lidocaine (2%) (Xylocaine) ',
       'Morphine (Morphine Sulfate) ', 'Norepinephrine (Levophed)',
       'Calcium Chloride (10%) ', 'Tetracaine',
       'Cyanokit (Hydroxocobalamin)',
       'Sodium Bicarbonate (4.2%) (Infant)'], dtype=object)

Confirmed that the unique Medication_Given_Description coun matches the Medication_Given_RXCUI_Code count of <b>32.</b>

In [16]:
#Confirming Tenure Months(unique) in sorted order
print('Tenure Months list =\n '+str(dfMedPatient.TenureMonths.sort_values().unique()))
##dfMedPatient.TenureMonths.sort_values().unique()

Tenure Months list =
 [-20 -11   3   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 

In [17]:
#Show number of unique values per column
dfMedPatient.nunique(dropna=False)

PatientId                       38384
PatientGender                       4
PatientGenderCode                   4
FRDPersonnelID                    843
FRDPersonnelGender                  2
ProviderGenderCode                  2
Medication_Given_RXCUI_Code        32
Medication_Given_Description       32
TenureMonths                      420
DispatchTime                    38378
FRDPersonnelStartDate             165
PersonelStartYear                  38
PersonelStartMonthYear            142
MedicationGivenYear                 4
dtype: int64

In [18]:
#Show number of nulls per column again, just to make sure
dfMedPatient.isnull().sum()

PatientId                        0
PatientGender                   32
PatientGenderCode                0
FRDPersonnelID                   0
FRDPersonnelGender               0
ProviderGenderCode               0
Medication_Given_RXCUI_Code      0
Medication_Given_Description     0
TenureMonths                     0
DispatchTime                     0
FRDPersonnelStartDate            0
PersonelStartYear                0
PersonelStartMonthYear           0
MedicationGivenYear              0
dtype: int64

Dispalying Time Traveler - Providers ( TenureMonths in future (<0 months) )

In [19]:
# showing dfMedPatient for those records that have a negative tenure ( time travellers)
dfMedPatient[(dfMedPatient['TenureMonths'] < 0)]

,PatientId,PatientGender,PatientGenderCode,FRDPersonnelID,FRDPersonnelGender,ProviderGenderCode,Medication_Given_RXCUI_Code,Medication_Given_Description,TenureMonths,DispatchTime,FRDPersonnelStartDate,PersonelStartYear,PersonelStartMonthYear,MedicationGivenYear
4207,547276,Male,2,6FB584B8-F37D-45E3-8FCC-C820ACF46816,Male,2,7806.0,Oxygen,-20,2018-03-17 10:43:35,2019-12-09,2019,2019-12,2018
16830,754276,Male,2,86D2C99E-9E01-E211-B5F5-78E7D18CFD3C,Female,1,7806.0,Oxygen,-11,2018-11-05 18:55:39,2019-10-29,2019,2019-10,2018


### Reduce Data

Going to drop the "time traveler" (negative tenure) rows

In [20]:
#Remove -ve Tenure for PatientProvider ( Removing time traveler records)
dfMedPatientReduced = dfMedPatient[(dfMedPatient['TenureMonths'] >= 0)]

#Calculate percentage of dataset remaining
ratio=len(dfMedPatientReduced)/len(dfMedPatient)*100
print("%.4f%% data remaining!" % ratio)

99.9968% remaining!


<B> Now adding a new column 'TenureYears' for further analytics </b>.Here we simply divide the TenureMonths by 12 and drop the remainder.
Calculation of Tenure in Months results in value that represents number of completed years of service. (e.g. 35 months -> 2 years, 36 months -> 3 years)

In [21]:
dfMedPatientReduced.loc[:,"TenureYears"]=np.floor(dfMedPatientReduced["TenureMonths"]/12)

C:\Users\anupd\Anaconda3\envs\emspe\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\anupd\Anaconda3\envs\emspe\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [ ]:
##dfMedPatientReduced.head()

In [ ]:
# Cross tabulation between  TenureMonths and Medications from 'dfMedPatientReduced' datarframe  -- Keeping this code line from -cross-validation/checking----
CrosstabResult=pd.crosstab(index=dfMedPatientReduced['TenureMonths'],columns=dfMedPatientReduced['Medication_Given_Description'])
##print(CrosstabResult)

Chi-square Test in Python can be done using the chi2_contingency() function from the scipy.stats module.

In [ ]:
# importing the required function
from scipy.stats import chi2_contingency

In [ ]:
# Cross tabulation between TenureMonths and Medications from 'dfMedPatientReduced' datarframe 
pd.crosstab(dfMedPatientReduced.Medication_Given_Description,dfMedPatientReduced.TenureMonths)

Chi-square test finds the probability of a Null hypothesis(H0).

 - Assumption(H0): The two variables are NOT related to each other
 - Result of Chi-Sq Test: if the Probability of H0 being True, The two variables are NOT related to each other
<p>It can help us to understand whether both the categorical variables are correlated with each other or not.

In [ ]:
## Performing Chi-sq test, to test the association between two variables, using the cross tab sequence
##ChiSquareResult=chi2_contingency(pd.crosstab( dfMedPatientReduced.TenureMonths, dfMedPatientReduced.Medication_Given_Description))
ChiSquareResult=chi2_contingency(pd.crosstab( dfMedPatientReduced.TenureMonths, dfMedPatientReduced.Medication_Given_RXCUI_Code))

In [ ]:
# P-Value is the Probability of H0 being True
# If P-Value>0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSquare Test is:', ChiSquareResult[1])

As evident, the <b>p-value is less than 0.05 </b>, hence we <b>reject the Null Hypothesis(H0) that the 'TenureMonths' of the Providers are not associated with the 'Medication_Given'</b>

As the P-value came lower than 0.05 in our result, hence H0 will not be accepted, which means the <b>variables 'TenureMonths' and 'Medication_Given' are correlated  to each other</b>. This is based on if two variables are correlated, then the P-value will come very close to zero, which in our case is : <b>4.5781793145246635e-174</b>

<b>Performing Chi-square test with TenureYears values </b>

In [ ]:
## Performing Chi-sq test with TenureYears, to test the association between two variables, using the cross tab sequence
ChiSquareResultforTYears=chi2_contingency(pd.crosstab( dfMedPatientReduced.TenureYears, dfMedPatientReduced.Medication_Given_RXCUI_Code))

In [ ]:
# P-Value is the Probability of H0 being True
# If P-Value>0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSquare Test( Teure Years) is:', ChiSquareResultforTYears[1])

<b>Now, further investigating relationship between the provider's TenureMonths and Medications Given follows </b>:........

Create a pivot-like dataframe using tenure, Medication Code and  Medication Description to get a Medication Given count break down by tenure and medication.

In [ ]:
## Creating pivot dataframe from'dfMedPatientReduced' 
dfMedPatientReduced_pvt = dfMedPatientReduced.groupby(['TenureMonths','TenureYears',
                                    'Medication_Given_RXCUI_Code',
                                    'Medication_Given_Description']).size().to_frame(name='Medication_Count').reset_index()

dfMedPatientReduced_pvt.shape

<b>What are counts of all 32 Medication Given overall? </b>

In [ ]:
dfMedPatientReduced_pvt.groupby(['Medication_Given_RXCUI_Code',
                          'Medication_Given_Description'])[['Medication_Count']].sum().nlargest(32,['Medication_Count'])

<b> Removing data where Medication_Given_Description = 'Oxygen' and 'Normal saline' from dfMedPatientReduced dataframe, before 2nd round of Chi-Square Test  </b>

In [ ]:
#Remove 'oxygen' for 'Medication_Given_Description'
dfMedPatientReduced_NoOxy=dfMedPatientReduced.loc[dfMedPatientReduced['Medication_Given_Description']!='Oxygen',:]

#size of remaining dataset 
len(dfMedPatientReduced_NoOxy)

In [ ]:
#Remove 'Normal saline' for 'Medication_Given_Description'
dfMedPatientReduced_NoOxySaline=dfMedPatientReduced_NoOxy.loc[dfMedPatientReduced_NoOxy['Medication_Given_Description']!='Normal saline',:]

#size of remaining dataset 
len(dfMedPatientReduced_NoOxySaline)

In [ ]:
# Now again creating Cross tabulation between TenureYears(this time) and Medications from 'dfMedPatientReduced_NoOxySaline' datarframe 
pd.crosstab(dfMedPatientReduced_NoOxySaline.Medication_Given_Description,dfMedPatientReduced_NoOxySaline.TenureYears)

In [ ]:
## Now Performing Chi-sq test, without 'oxygen' and 'Normal saline' and 'TenureYears' to test the association between two variables, using the cross tab sequence
ChiSquareResult_NoOxySaline=chi2_contingency(pd.crosstab(dfMedPatientReduced_NoOxySaline.TenureYears, dfMedPatientReduced_NoOxySaline.Medication_Given_Description))

In [ ]:
# P-Value is the Probability of H0 being True
# If P-Value>0.05 then only we Accept the assumption(H0)
 
print('The P-Value of the ChiSquare Test is:', ChiSquareResult_NoOxySaline[1])

Again, we see the <b>p-value is less than 0.05 </b>, hence we <b>reject the Null Hypothesis(H0) that the 'TenureMonths' of the Providers are not associated with the 'Medication_Given'</b>

<b>Using dfMedPatientReduced_pvt for further analysis</b>

In [ ]:
## Changing 'Medication_Given_RXCUI_Code' from Numerical(float) to int64 for consistency
dfMedPatientReduced_pvt['Medication_Given_RXCUI_Code'] = dfMedPatientReduced_pvt['Medication_Given_RXCUI_Code'].apply(np.int64) 

In [ ]:
#Create the MedicationOrder array from the dataframe
MedicationOrder=np.array(dfMedPatientReduced["Medication_Given_Description"])
MedicationOrder

In [ ]:
## Lowest 3 tenure months Medication given, count break down by tenuremonths
dfMedPatientReduced_pvt.head(3)

In [ ]:
## Top 3 tenuremonths for Medication given, count break down by tenuremonths
dfMedPatientReduced_pvt.tail(3)

Generate a hex bin plot using tenure, medication given code, and medication count from dfMedPatient_pvt as the x, y and C values, respectively.

In [ ]:
# in order to get the medication given code to show in the y-axis, need to format the values as strings instead of numbers
hb = dfMedPatientReduced_pvt.plot.hexbin(
                       x='TenureYears', 
                       y='Medication_Given_RXCUI_Code', 
                       C='Medication_Count',
                       reduce_C_function=np.sum,
                       gridsize=110,
                       cmap="nipy_spectral_r",
                       xlabel="Provider Tenure(Years)",
                       ylabel="Medication Code",
                       title="Provider Tenure and Medication Given",
                       figsize=(16,14)
                       ##sharex=False
)
plt.gca().yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter('{x:.0f}'))
plt.show

The hex bin plots of dfMedPatientReduced_pvt shows the use of 3 attributes from the dataframe. Using x, y, and C means that the reduce_C_function now comes into play and it is the reason for all of the white showing - it is where there is no sum to show.

Using dfMedPatientReduced_pvt, generate descriptive statistics about the tenure, medicine count, and medication given description attributes.

In [ ]:
dfMedPatientReduced_pvt[['TenureYears','Medication_Count']].describe()

In [ ]:
plt.figure(figsize=(18,20))
##sns.set_theme(style="whitegrid")
sns.set(style="darkgrid")
sns.set(font_scale = 2)
sns.violinplot("TenureYears","Medication_Given_Description",
               data=dfMedPatientReduced[dfMedPatientReduced.TenureYears < 40], 
               scale='width', linewidth=2.5,
              order=MedicationGivenOrder, cut=0, palette="muted", inner="quartile");
plt.title('Medications Given by Provider Tenure')
plt.xlabel("Tenure (Years)", size=16)
plt.ylabel("Medication Description", size=16)
##plt.savefig("barplot_MedicationGiven_byProviderTenure.png")

In [ ]:
##plt.title('Medication Given vs Tenure Violinplot')
plt.figure(figsize=(16,18))
sns.set(style="darkgrid")
sns.set(font_scale = 2)
sns.violinplot(x="TenureYears",y="Medication_Given_Description",
               data=dfMedPatientReduced, hue="FRDPersonnelGender", split=True, dodge =True,
               scale='area',
              order=MedicationGivenOrder, cut=0, inner="quartile", height=15, aspect=.8, palette={"Male": "b", "Female": "y"});
plt.title('Medications Given by Provider Tenure and Gender')
plt.xlabel("Tenure (Years)", size=16)
plt.ylabel("Medication Description", size=16)
##plt.savefig("barplot_MedicationGiven_byProviderTenureGender.png")

<b>Now, Creating violin plot after removing Oxygen' & 'Normal Saline' with high skewness for comparision</b>

In [ ]:
## Using dataframe 'dfMedPatientReduced_NoOxySaline' removing medicine 'Oxygen' & 'Normal Saline' for violin plot 

plt.figure(figsize=(16,18))
sns.set(style="darkgrid")
sns.set(font_scale = 2)
sns.violinplot(x="TenureYears",y="Medication_Given_Description",
               data=dfMedPatientReduced_NoOxySaline, hue="FRDPersonnelGender", split=True, dodge =True,
               scale='count',
              order=MedicationGivenOrderWithoutOxySaline, cut=0, palette={"Male": "b", "Female": "y"}, inner="quartile", height=15, aspect=.8);
plt.title('Medications Given by Provider Tenure(years) - No Oxygen/Saline')
plt.xlabel("Tenure (Years)", size=16)
plt.ylabel("Medication Description", size=16)
##plt.savefig("violinplot_MedicationGiven_byProvTenureGender.png")

Now,looking for more details on the data frame

In [ ]:
dfMedPatientReduced_pvt[['Medication_Given_Description']].describe(include='all')

Generate histograms of the tenure and medication given using all records to see frequency from dfMedPatient_pvt

In [ ]:
## histogram of the tenure
##dfMedPatientReduced.hist(column='TenureYears',bins=40)
dfMedPatientReduced.hist(column='TenureYears',bins=int(dfMedPatientReduced["TenureYears"].max()))

Finding the distributions for tenure years of the providers(also gender)

In [ ]:
sns.set_palette("RdBu")
sns.violinplot(x="TenureYears",y="FRDPersonnelGender",
               data=dfMedPatientReduced,
               split=True,
               scale='count', cut=-0, palette={"Male": "b", "Female": "y"}, bw=0.04);
sns.despine(left=True)
plt.title('Provider Tenure and Gender - Distribution', size=20)
plt.xlabel("Tenure Year", size=16)
plt.ylabel("Provider Gender", size=16)

<b>Reviewing distribution of Medication_Given_Description </b>

In [ ]:
## bar plot for medication given distribution 
fig=plt.figure(figsize=(20,14))
value_counts=dfMedPatientReduced['Medication_Given_Description'].value_counts()
df_GivenMedicationByProviders=pd.DataFrame(value_counts)
df_GivenMedicationByProviders.reset_index(inplace=True)
df_GivenMedicationByProviders.columns=["Medication_Given_Description","Count"]

#Bar chart
ax1=plt.subplot()
ax1.barh(df_GivenMedicationByProviders['Medication_Given_Description'],width=df_GivenMedicationByProviders['Count'],color='purple')
ax1.set_xlabel(' Medications Given - Number ')
ax1.grid(axis='x', alpha=.75, which='both')

ax1.set_xticks(minor=True, ticks=[100,200,300,400,600,700,800,900,1100, 1200,1300,1400])
ax1.set_xticks(minor=False, ticks=[500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 10500])
                                  
fig.suptitle('Medication Given - Distribution')
plt.show()
##plt.savefig("Barplot_MedicationGiven_Distribution.png")

In [ ]:
## !! Check, if this is valid !!! histogram of the 'Medication_Count' (by tenure)
##dfMedPatientReduced_pvt.hist(column='Medication_Count', bins=40)

Please remember there are some "time traveler" tenure values (2 records) where the porvider start date is after the dispatch date resulting in a negative value for the calculated provider tenure. Those records were removed for analysis.

<b>What are the top 10 Medication Given overall? </b>

In [ ]:
dfMedPatientReduced_pvt.groupby(['Medication_Given_RXCUI_Code',
                          'Medication_Given_Description'])[['Medication_Count']].sum().nlargest(10,['Medication_Count'])

Based on the tenure histogram, the highest tenure count  falls between the 0 years (< 1 year) and 20 years value(mean tenure 13.37 years) . Now going to look at provider tenure values less than or equal to 20 years tenure to see what can be found around these "high" range.

In [ ]:
# how many records from dfMedPatientReduced_pvt will be used limiting the tenure to 100 months or less
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)].shape

<b>Generate the respective descriptive statistics and histograms. </b>

In [ ]:
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)][['TenureYears','TenureMonths','Medication_Count']].describe()

In [ ]:
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)][['Medication_Given_Description']].describe(include='all')

In [ ]:
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)].hist(column='Medication_Given_RXCUI_Code', bins=40)

A comparison of the medication given histograms for all dfMedPatientReduced_pvt rows and the subset limited to tenure of 20 years or less shows the distribution of medication given is seemingly distributed evenly except for certain medications.

The mean of the tenure values for those providers with 20 years or less, is <b>9.46 </b> years, as seen above earlier.

<b> What are the top 10 procedures by count for medication given by providers with 20 or less years of tenure? </b>

In [ ]:
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)].groupby(['Medication_Given_RXCUI_Code',
     'Medication_Given_Description'])[['Medication_Count']].sum().nlargest(10,['Medication_Count'])

In [ ]:
# create a dataframe for medication given by providers with 20 or less years of tenure for repeated use in analysis
dfMedPatientReduced_pvt_lte20Yrs= dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] <= 20)]
dfMedPatientReduced_pvt_lte20Yrs.shape

<b>What are the top 10 procedures by count for procedures performed by providers with more than 20 or more years of tenure? </b>

In [ ]:
dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] > 20)].groupby(['Medication_Given_RXCUI_Code',
     'Medication_Given_Description'])[['Medication_Count']].sum().nlargest(10,['Medication_Count'])

In [ ]:
# create a dataframe for medication given by providers with more than 20 years of tenure for repeated use in analysis
dfMedPatientReduced_pvt_gt20Yrs= dfMedPatientReduced_pvt[(dfMedPatientReduced_pvt['TenureYears'] > 20)]
dfMedPatientReduced_pvt_gt20Yrs.shape

<b>Generate the respective descriptive statistics and histograms for medication given by providers with more than 20 years of tenure. </b>

In [ ]:
# describe for dataframe dfMedPatientReduced_pvt_gt20Yrs
dfMedPatientReduced_pvt_gt20Yrs[['TenureYears','TenureMonths','Medication_Count']].describe()

In [ ]:
# describe for 'Medication_Given_Description'
dfMedPatientReduced_pvt_gt20Yrs[['Medication_Given_Description']].describe(include='all')

The medication histogram for providers with 20 years or less and greater than 20 years continues to display a similar distribution pattern of medication given (even scales are less)

In [ ]:
# show procedure histograms from all tenures, tenure < = 20 years and  tenure > 300 years side by side

fig=plt.figure(figsize=(12,4))

# all tenure
ax1=plt.subplot(121)
ax1.hist(x=dfMedPatientReduced_pvt['Medication_Given_RXCUI_Code'],bins= 40, alpha = 0.5)
ax1.set_xlabel('Medication Given Code')
ax1.set_ylabel('Medication Count')
ax1.set_title('Med. Given by All Providers')
ax1.ticklabel_format(style='plain')
ax1.grid()
plt.xticks(rotation=45)


# lte >=20 years tenure
ax2=plt.subplot(122)
ax2.hist(x=dfMedPatientReduced_pvt_lte20Yrs['Medication_Given_RXCUI_Code'], bins=40, color='red')
ax2.set_xlabel('Medication Given Code')
ax2.set_ylabel('Medication Count')
ax2.set_title('Med. Given by Tenure < 20 years')
ax2.ticklabel_format(style='plain')
ax2.grid()
plt.xticks(rotation=45)

# show the first 2 plots
plt.show()

fig=plt.figure(figsize=(12,4))

# gt > 30 years tenure
ax1=plt.subplot(121)
ax1.hist(x=dfMedPatientReduced_pvt_gt20Yrs['Medication_Given_RXCUI_Code'], bins=40, color='purple')
ax1.set_xlabel('Medication Given Code')
ax1.set_ylabel('Medication Count')
ax1.set_title('Med. Given by Providers with tenure > 20 years')
ax1.ticklabel_format(style='plain')
ax1.grid()
plt.xticks(rotation=45)

# show all the plots
plt.show()


Now showing the histograms as overlays so thie comparision can be seen  on the same scale

In [ ]:
## overlays histograms 
fig=plt.figure(figsize=(16,8))

ax1=plt.subplot(121)
ax1.hist(x=dfMedPatientReduced_pvt['Medication_Given_RXCUI_Code'],bins= 40, alpha = 0.5, label='all tenures')
ax1.hist(x=dfMedPatientReduced_pvt_lte20Yrs['Medication_Given_RXCUI_Code'], bins=40, color='red', label='<= 20 years')
ax1.hist(x=dfMedPatientReduced_pvt_gt20Yrs['Medication_Given_RXCUI_Code'], bins=40, color='purple', label='> 20 years')
ax1.set_xlabel('Medication Given Code')
ax1.set_ylabel('Medication Count')
ax1.set_title('Medication Given by Providers Tenure - Overlay')
ax1.ticklabel_format(style='plain')
ax1.grid()
plt.xticks(rotation=45)
plt.legend(loc='upper right')
plt.show()


<b>Now looking for the following medications which has adminstered by providers with specific tenures </b>
  (Medication Name    Code     Group)
 - Magnesium Sulfate (50%) - 6585 - Multi-Use
 - Tranexamic Acid (TXA) - 10691 - Hemorrhaging
 - Lidocaine (2%) (Xylocaine) - 6387 - Pain
 - Morphine (Morphine Sulfate) - 7052 - Pain
 - Norepinephrine (Levophed) - 328853 - Hypotension
 - Calcium Chloride (10%) - 1901 - Cardiac
- Tetracaine - 91189 - Eye Pain
 - Cyanokit (Hydroxocobalamin) - 5514 - Poisoning



In [ ]:
## Checking info about dfMedPatientReduced for data format check
dfMedPatientReduced.head()

In [ ]:
## Checking 'data types' of dataframe dfMedPatientReduced
##dfMedPatientReduced.dtypes

In [ ]:
# Now converting 'Medication_Given_RXCUI_Code' from float to int for consistency
dfMedPatientReduced['Medication_Given_RXCUI_Code'] = dfMedPatientReduced['Medication_Given_RXCUI_Code'].apply(np.int64)
dfMedPatientReduced.dtypes

In [ ]:
## Filtering records by Medication_Given_RXCUI_Code
Medication_CodeList_toInclude = [6585, 10691,6387,7052, 328853, 1901, 91189, 5514 ]

In [ ]:
## Keep the records where the Medication_Given_RXCUI_Code is in the list:
dfMedPatientReduced_Filtered = dfMedPatientReduced[dfMedPatientReduced['Medication_Given_RXCUI_Code'].isin(Medication_CodeList_toInclude)]

In [ ]:
###dfMedPatientReduced_Filtered = dfMedPatientReduced[dfMedPatientReduced.Medication_Given_Description.isin(["Glucagon (Glucagen)","Magnesium Sulfate (50%)", "Tranexamic Acid (TXA)", "Lidocaine (2%) (Xylocaine)","Morphine (Morphine Sulfate)", "Norepinephrine (Levophed)","Calcium Chloride (10%)", "Tetracaine","Cyanokit (Hydroxocobalamin)"])]
##dfMedPatientReduced_Filtered.head()

In [ ]:
dfMedPatientReduced_Filtered.size

In [ ]:
#Put the value_counts into a dataframe for later use
value_countsMedGivenReducedList=dfMedPatientReduced_Filtered['Medication_Given_Description'].value_counts()
df_ReducedListMedicationGivenDescription=pd.DataFrame(value_countsMedGivenReducedList)
df_ReducedListMedicationGivenDescription.reset_index(inplace=True)
df_ReducedListMedicationGivenDescription.columns=["Medication_Given_Description","Count"]
df_ReducedListMedicationGivenDescription.set_index(keys=["Medication_Given_Description"])
df_ReducedListMedicationGivenDescription

In [ ]:
#Create the MedicationGivenOrder(Reduce list) array from the newly created dataframe
ReducedList_MedicationGivenOrder=np.array(df_ReducedListMedicationGivenDescription["Medication_Given_Description"])
##ReducedList_MedicationGivenOrder

In [ ]:
## Creating pivot dataframe from'dfMedPatientReduced' by Provider start Year
dfMedPatientReducedByStartYear_FilteredPvt = dfMedPatientReduced_Filtered.groupby(['PersonelStartYear','Medication_Given_RXCUI_Code',
     'Medication_Given_Description']).size().to_frame(name='MedGiven_Count').reset_index()

In [ ]:
dfMedPatientReducedByStartYear_FilteredPvt.size

In [ ]:
## Creating pivot dataframe from'dfMedPatientReduced' group by Medication GivenYear
dfMedPatientReducedByMedGivenYear_FilteredPvt = dfMedPatientReduced_Filtered.groupby(['MedicationGivenYear','Medication_Given_RXCUI_Code',
     'Medication_Given_Description']).size().to_frame(name='MedGiven_Count').reset_index()

In [ ]:
dfMedPatientReducedByMedGivenYear_FilteredPvt.size

In [ ]:
## Creating pivot dataframe from'dfMedPatientReduced' for Medication given by tenure years
dfMedPatientReducedByMedGivenTenureYears_FilteredPvt = dfMedPatientReduced_Filtered.groupby(['TenureYears','Medication_Given_RXCUI_Code',
     'Medication_Given_Description']).size().to_frame(name='MedGiven_Count').reset_index()

In [ ]:
dfMedPatientReducedByMedGivenTenureYears_FilteredPvt

In [ ]:
##sns.color_palette("RdBu", 10)
sns.color_palette("BuPu", 10)

plt.figure(figsize=(18,12))
sns.set_color_codes("pastel")
sns.set_context("paper", font_scale=1.5) 
ax = sns.barplot(y="MedGiven_Count", x="PersonelStartYear",
    data=dfMedPatientReducedByStartYear_FilteredPvt,
    ci=None, hue= "Medication_Given_Description")
ax.set_xlabel("MedGiven_Count")
plt.rcParams["axes.labelsize"] = 6
plt.ylabel("Medication Given Count", size=15)
plt.xlabel("Personel Start Year", size=15)
plt.title("Medication Administered - by Provider's Start Year(for filtered 8 medications only)", size=16)
plt.legend(loc='upper left')
##plt.savefig("barplot_MedicationGiven_byPersonalStartYear.png")


Find Medication Administered by Medication given Year( for filtered 8 medications only )

In [ ]:
dfMedPatientReducedByMedGivenYear_FilteredPvt

In [ ]:
plt.figure(figsize=(12,10))
sns.set(style="darkgrid")
sns.set(font_scale = 2)
sns.violinplot("MedGiven_Count","Medication_Given_Description",
               data=dfMedPatientReducedByMedGivenYear_FilteredPvt, 
               scale='count', linewidth=2.5,
              order=ReducedList_MedicationGivenOrder, cut=0, palette="muted");
plt.title('Medications Given Count (8 Medications)')
plt.xlabel("Medication Count", size=16)
plt.ylabel("Medication Description", size=16)
##plt.savefig("Violinplot_MedicationGiven_byPersonalMedGivenYear.png")

In [ ]:
plt.figure(figsize=(16,12))
sns.set_color_codes("pastel")
##sns.set_color_codes("dark")
##sns.set_color_codes("muted")

sns.set_context("paper", font_scale=1.5) 
ax = sns.barplot(y="MedGiven_Count", x="MedicationGivenYear",
    data=dfMedPatientReducedByMedGivenYear_FilteredPvt,
    ci=None, hue= "Medication_Given_Description")
ax.set_xlabel("MedGiven_Count")
plt.rcParams["axes.labelsize"] = 8
plt.ylabel("Medication Given Count", size=15)
plt.xlabel("Medication Given Year", size=15)
plt.title("Medication Administered - per Year(for filtered 8 medications)", size=16)
plt.legend(loc='upper right')
##plt.savefig("barplot_MedicationGivenByYear.png")

In [ ]:
# Now converting 'TenureYears' from float to int for better display
dfMedPatientReducedByMedGivenTenureYears_FilteredPvt['TenureYears'] = dfMedPatientReducedByMedGivenTenureYears_FilteredPvt['TenureYears'].apply(np.int64)
dfMedPatientReducedByMedGivenTenureYears_FilteredPvt.dtypes

In [ ]:
plt.figure(figsize=(16,12))
##sns.set_color_codes("pastel")
sns.set_color_codes("bright")

sns.set_context("paper", font_scale=1.5) 
ax = sns.barplot(y="MedGiven_Count", x="TenureYears",
    data=dfMedPatientReducedByMedGivenTenureYears_FilteredPvt,
    ci=None, hue= "Medication_Given_Description" )
ax.set_xlabel("MedGiven_Count")
plt.rcParams["axes.labelsize"] = 8
plt.ylabel("Medication Given Count", size=15)
plt.xlabel("Provider Tenure Year", size=15)
plt.title("Medication Administered - by Provider'ss Tenure Year(for filtered 8 medications)", size=16)
plt.legend(loc='upper right')
##plt.savefig("barplot_MedicationGiven_byPersonalStartYear.png")

#### Now lets add the MedicationGroup column to the main dataset to group its value and further analyze with provider tenure

In [ ]:
#Import Medication Group
localFolder='../data/02_intermediate/' #If run from Git Hub project folder structure, script must be in "notebooks" folder
dfmedicationClasses=pd.read_csv(localFolder+'MedicationGroups.csv',usecols=['Medication_Given_RXCUI_Code','MedicationGroup'])

In [ ]:
dfmedicationClasses.head(5)

In [ ]:
#Add a column for 'Medication Group' in dataframe dfMedPatientReduced to create the new column
##dfMedPatientReduced.merge(dfmedicationClasses,how='left',on='Medication_Given_RXCUI_Code')

#Add a column for 'Medication Group' in dataframe dfMedPatientReduced to create the new column
dfMedPatientReduced_withMedGroup = pd.merge(dfMedPatientReduced,dfmedicationClasses,how='left',on='Medication_Given_RXCUI_Code')

#drop the Count column as it is not needed here
##dfMedPatientReduced_withMedGroup.drop(columns=['Count'], inplace=True)
print("There are {} records in dfMedPatientReduced".format(len(dfMedPatientReduced)))
print("There are {} records in dfMedPatientReduced_withMedGroup".format(len(dfMedPatientReduced_withMedGroup)))
##dfMedPatientReduced_withMedGroup.head()

#### Medication Groups and Medication Given List: 
1. <b>Allergic Reaction</b>
 - Diphenhydramine (Benadryl)
 - Epinephrine 1 MG/ML (1:1,000)
2. <b>Behavioral Control & Pain </b>
 - Ketamine (Ketalar)
3. <b>Cardiac</b>
 - Acetylsalicylic Acid (Aspirin, ASA) 
 - Adenosine (Adenocard)
 - Amiodarone (Cordarone)
 - Atropine (Atropine Sulfate) 
 - Calcium Chloride (10%) 
 - Epinephrine 0.1 MG/ML (1:10,000)
 - Nitroglycerin
4. <b>Diabetic</b>
 - Dextrose 10 % (D10)
 - Dextrose 50 % (D50)
 - Glucagon (Glucagen)
 - Glucose (Oral)
5. <b>Eye Pain</b>
 - Tetracaine
6. <b>General</b>
 - Normal saline
 - Oxygen
7. <b>Hemorrhaging </b>
 - Tranexamic Acid (TXA)
8. <b>Hypotension </b>
 - Norepinephrine (Levophed)
9. <b>Multi-Use </b>
 - Magnesium Sulfate (50%) 
 - Methylprednisolone (Solu-Medrol) 
 - Sodium Bicarbonate (4.2%) (Infant)
 - Sodium Bicarbonate (8.4%)
10. <b>Nausea & Vomiting </b>
 - Ondansetron (Zofran)
11. <b>Overdose </b>
 - Naloxone (Narcan)
12. <b>Pain </b>
 - Fentanyl Citrate (Sublimaze) 
 - Lidocaine (2%) (Xylocaine) 
 - Morphine (Morphine Sulfate) 
13. <b>Poisoning </b>
 - Cyanokit (Hydroxocobalamin)
14. <b>Respiratory </b>
 - Albuterol (Ventolin) 
 - Ipratropium Bromide(Atrovent)
15.<b>Seizures & Sedation </b>
 - Midazolam (Versed)

In [ ]:
plt.figure(figsize=(16,18))
##sns.set_theme(style="whitegrid")
sns.set(style="darkgrid")
sns.set(font_scale = 2)
sns.violinplot("TenureYears","MedicationGroup",
               data=dfMedPatientReduced_withMedGroup, hue="FRDPersonnelGender",
               scale='area', linewidth=2.2,
               inner="box",
              cut=0, palette={"Male": "b", "Female": "y"});
plt.title('Medications Group by Provider Tenure(years)')
plt.ylabel("Medication Group", size=18)
plt.xlabel("Provider Tenure Year", size=18)
##sns.despine(left=True)
##plt.savefig("ViolinPlot_MedicationGroup_byProviderTenure.png")

### Findings
Question: Is there a relationship between category(name) of medication given and the provider tenure?
<p>Answer: In my initial analysis ( using chi-square Chi-square Test of Independence)  it is suggested to be some indicaton of a relationship between medication given and the provider tenure. I further analyzed the relationship between medication given and the provider tenure using various data analytical and visualization methods. In some cases I find there are relationship between medication given and the provider tenure. However, we do not see anomaly associated with provider tenure and the medication given.

<b>Conclusions / Observations </b>
 - The distribution of medication given is nominally similar across all tenures (by year), with some slight variation.
 - The distribution of tenures (by year) is nominally similar across all medication given, with some slight variation.
 - These distributions are consistent with the distributions of their underlying factors.
 - Certain medications groups/types are administered more by a certain Provider Tenure. However no anomaly of medication administration was found.
 - Certain of Medications Type(Name) administered can be relate to the Provider's Start year/tenure.
 - Certain number of medications types ( among 8 medications reviewed) were given more(or less) on a certain year.
<li><b> More...</b>
